### json 불러와서 저장하기

In [39]:
import numpy as np
import pandas as pd
import json
import jpype
import glob
from random import shuffle
from bs4 import BeautifulSoup as bs

data_path = "data"
#data_path = "data(test)"

file_list = glob.glob("%s/*.json" % data_path)

json_train=[]

shuffle(file_list)
for json_file_name in file_list:
    json_file = json.loads(open(json_file_name).read())
    json_train += json_file["articles"]

### json에서 author, author 형태소, forumid 추출

In [40]:
from konlpy.tag import Mecab

mecab = Mecab()

labeled_train = []

for cnt, article in enumerate(json_train):
    if cnt % 10000 == 0:
        print cnt

    labeled_train.append({
            "istroll": article["is_troll"],
            "author": article["author"],
            "author_pos": ["%s_%s" % (word, pos) for word, pos in mecab.pos(article["author"])],
            "forumid": article["forumid"],
            "pk": article["pk"]
        })

labeled_train = pd.DataFrame.from_dict(labeled_train)
labeled_train = labeled_train.set_index('pk')

labeled_train

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000


,author,author_pos,forumid,istroll
pk,,,,
64993,메론바흐규,"[메론_NNP, 바흐_NNP, 규_NNG]",요리밭,False
64994,메론바흐규,"[메론_NNP, 바흐_NNP, 규_NNG]",동물식물밭,False
64995,Eliciel,[Eliciel_SL],잡담밭,False
64996,김카디,"[김_NNP, 카디_NNG]",잡담밭,False
64997,한줌의흙,"[한줌_NNG, 의_JKG, 흙_NNG]",IT밭,False
64998,라트,[라트_NNP],음악/공연밭,False
64999,민영화는대체어떤냔이얌?,"[민영화_NNG, 는_JX, 대체_MAG, 어떤_MM, 냔_ETM, 이얌_NNG, ...",잡담밭,False
65000,LEOD,[LEOD_SL],잡담밭,False
65001,Eliciel,[Eliciel_SL],잡담밭,False


### forumid와 author을 label encoder로 숫자로 바꿔줌

In [41]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

labeled_train["forumid"] = le.fit_transform(labeled_train["forumid"])
labeled_train["author"] = le.fit_transform(labeled_train["author"])

labeled_train

,author,author_pos,forumid,istroll
pk,,,,
64993,12009,"[메론_NNP, 바흐_NNP, 규_NNG]",25,False
64994,12009,"[메론_NNP, 바흐_NNP, 규_NNG]",14,False
64995,1011,[Eliciel_SL],31,False
64996,6770,"[김_NNP, 카디_NNG]",31,False
64997,25906,"[한줌_NNG, 의_JKG, 흙_NNG]",1,False
64998,10909,[라트_NNP],29,False
64999,12784,"[민영화_NNG, 는_JX, 대체_MAG, 어떤_MM, 냔_ETM, 이얌_NNG, ...",31,False
65000,1394,[LEOD_SL],31,False
65001,1011,[Eliciel_SL],31,False


###bag of words

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

def make_bag_of_words(labeled_train, max_features, col_name):
    vectorizer = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None,
                                stop_words = None, max_features=max_features)
    
    sentences = labeled_train[col_name].apply(lambda x:" ".join(x))
    train_data_features = vectorizer.fit_transform(sentences).toarray()

    col = ["bow_%s_%s" % (col_name, data) for data in vectorizer.get_feature_names()]
    #col = ["bow%s%s" % (col_name, str(i)) for  in range(0, max_features)]
    df_bow = pd.DataFrame(train_data_features, columns = col, index=labeled_train.index)
    
    labeled_train = pd.concat([labeled_train, df_bow],axis=1)
    
    return labeled_train

##LDA

In [43]:
import gensim
from gensim import corpora, models
from gensim.corpora import TextCorpus, MmCorpus, Dictionary
from nltk.corpus import stopwords
from optparse import OptionParser

def make_lda(train, keep_n, num_topics, col_name):

    data = train[col_name]

    dictionary = corpora.Dictionary(data)
    dictionary.filter_extremes(keep_n=keep_n)

    corpus = [dictionary.doc2bow(text) for text in data]

    print("Make Lda..")

#    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1,distributed = True)
    lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, chunksize=1000, passes=1)
    
    num = len(train)    
    df = []
    
    for i in range(0,num):
        if i % 10000 == 0:
            print(i)
        
        temp = [i[1] for i in lda.get_document_topics(corpus[i],minimum_probability=0)]
        #temp = max(lda.get_document_topics(corpus[i]),key=lambda item:item[1])[0]
        df.append(temp)
    
    col = ["lda_%s_%d" % (col_name, data) for data in range(0, num_topics)]
    df = pd.DataFrame(df, columns = col)
    df.index = train.index
    
    train = pd.concat([train, df], axis=1)
        
    return train

##Word2Vec

In [45]:
from gensim.models import word2vec

def makeFeatureVec(words, model, num_features):
    
    featureVec = np.zeros((num_features,), dtype = "float32")
    
    nwords = 0
    
    index2word_set = set(model.index2word)
    
    for word in words:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])
    
    featureVec = np.divide(featureVec, nwords)
    
    return featureVec

def getAvgFeatureVecs(texts, model, num_features):
    
    counter = 0
    
    textFeatureVecs = np.zeros((len(texts), num_features), dtype = "float32")
    
    for i, text in enumerate(texts):
        
        if i % 10000 == 0:
            print(i)
        
        textFeatureVecs[i] = makeFeatureVec(text , model, num_features)
        
    return textFeatureVecs

def make_word2vec(train, col_name):
    num_features = 300
    min_word_count = 40
    num_workers = 4
    context = 10
    downsampling = 1e-3
    
    sentences = " ".join(train[col_name].apply(lambda x:" ".join(x)))
    
    model = word2vec.Word2Vec(sentences, workers = num_workers, size = num_features,\
                             min_count = min_word_count, window = context, sample = downsampling)
    
    train_feature = getAvgFeatureVecs(train[col_name].apply(lambda x:" ".join(x)), model, num_features)
    train_feature = pd.DataFrame(train_feature, index = train.index)
    
    train = pd.concat([train, train_feature], axis = 1)
    
    return train

In [46]:
max_features = 1000
# #%time labeled_train = make_lda(labeled_train, 1000, 200, "author_pos")
# #%time labeled_train = make_bag_of_words(labeled_train, 1000, "author_pos")

%time labeled_train = make_word2vec(labeled_train, "author_pos")
labeled_train

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
CPU times: user 36.6 s, sys: 12.1 s, total: 48.8 s
Wall time: 40.3 s


,author,author_pos,forumid,istroll,0,1,2,3,4,5,...,290,291,292,293,294,295,296,297,298,299
pk,,,,,,,,,,,,,,,,,,,,,
64993,12009,"[메론_NNP, 바흐_NNP, 규_NNG]",25,False,0.000541,3.729013e-04,0.000116,-0.000742,0.000196,-0.000427,...,-0.000439,-0.000031,-0.000133,-0.000376,-0.000073,0.000086,0.000620,0.000562,-0.000226,-0.000226
64994,12009,"[메론_NNP, 바흐_NNP, 규_NNG]",14,False,0.000541,3.729013e-04,0.000116,-0.000742,0.000196,-0.000427,...,-0.000439,-0.000031,-0.000133,-0.000376,-0.000073,0.000086,0.000620,0.000562,-0.000226,-0.000226
64995,1011,[Eliciel_SL],31,False,0.000507,-2.212759e-04,0.000365,-0.000637,-0.000314,0.000386,...,-0.000495,0.000413,-0.000167,0.000196,0.000240,-0.000045,0.000748,0.000630,-0.000431,0.000209
64996,6770,"[김_NNP, 카디_NNG]",31,False,0.000420,2.417436e-04,0.000034,-0.000680,0.000279,-0.000465,...,-0.000665,-0.000185,-0.000049,-0.000528,-0.000078,-0.000069,0.000348,0.000794,-0.000183,-0.000078
64997,25906,"[한줌_NNG, 의_JKG, 흙_NNG]",1,False,0.000484,8.960303e-04,-0.000198,-0.000855,0.000376,-0.000118,...,-0.000823,-0.000205,-0.000139,-0.000719,-0.000057,0.000191,0.000247,0.000526,-0.000413,-0.000166
64998,10909,[라트_NNP],29,False,0.000840,8.557888e-05,0.000214,-0.000911,0.000500,-0.000093,...,-0.000535,-0.000387,-0.000291,-0.000223,0.000039,-0.000068,0.000646,0.000845,-0.000506,-0.000256
64999,12784,"[민영화_NNG, 는_JX, 대체_MAG, 어떤_MM, 냔_ETM, 이얌_NNG, ...",31,False,0.000403,5.633188e-04,-0.000193,-0.000720,0.000123,-0.000073,...,-0.000267,0.000184,-0.000265,-0.000529,0.000088,-0.000192,0.000233,0.000492,-0.000211,-0.000148
65000,1394,[LEOD_SL],31,False,0.000021,-4.420975e-04,0.000078,-0.000722,-0.000036,0.000465,...,-0.000343,0.000427,-0.000080,-0.000335,0.000951,0.000016,0.000702,-0.000113,-0.000028,0.000263
65001,1011,[Eliciel_SL],31,False,0.000507,-2.212759e-04,0.000365,-0.000637,-0.000314,0.000386,...,-0.000495,0.000413,-0.000167,0.000196,0.000240,-0.000045,0.000748,0.000630,-0.000431,0.000209


### predictor, model 생성

In [49]:
from sklearn.ensemble import RandomForestClassifier

label = 'istroll'
pre = labeled_train.columns.drop(['author_pos', 'author', label])
labeled_train = labeled_train.fillna(0)

model = RandomForestClassifier(n_estimators=10, n_jobs=6)

###cross validation

In [50]:
from sklearn import cross_validation

cv_value = 3

scores = cross_validation.cross_val_score(model, labeled_train[pre], labeled_train[label], cv=cv_value, scoring="roc_auc")

cv_result = scores.mean()

print(cv_result)

0.869585615353


## Result

 * author를 Label Encoder로 변환: 0.67331271058
 * author의 morphs를 Bag of Words(feature = 1,000 개)로 변환: 0.733333689652
 * author의 pos를 Bag of Words(feature = 1,000 개)로 변환: 0.769833271386
 * 데이터를 shuffle하고 pos를 Bag of Words(feautre = 1,000개)로 변환: 0.81172740086
 
 
 * author의 pos를 LDA로 변환 (num_topics = 200, keep_n = 1,000) : 0.748718319834
 * 데이터를 shuffle하고 author의 pos를 LDA로 변환 (num_topics = 200, keep_n = 1,000) : 0.772313052394
 
 * 데이터 shuffle 후 bag of words(num_features = 300) : 0.869585615353

In [51]:
import pickle

pre = labeled_train.columns.drop(['author_pos', 'author'])

pickle.dump(labeled_train[pre], open("author, forum_%d.p" % max_features, "wb"), protocol = pickle.HIGHEST_PROTOCOL)